## Player Types For Clustering ##

```
1) Box to Box Midfielder
2) Holding Midfielder
4) Advanced Playmaker
5) Deep Lying Playmaker
6) Winger
7) False 9
8) Striker 
9) Target Forward
10) Center-Back
11) Wing-Back
12) Full-Back
13) GoalKeeper
```

In [84]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from time import time
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline

In [17]:
df = pd.read_csv('world_cup_comparisons.csv')

In [18]:
df.head()

,player,season,team,goals_z,xg_z,crosses_z,boxtouches_z,passes_z,progpasses_z,takeons_z,progruns_z,tackles_z,interceptions_z,clearances_z,blocks_z,aerials_z,fouls_z,fouled_z,nsxg_z
0,Cristian Pavón,2018,Argentina,-0.42,-0.55,0.08,-0.24,-0.61,-0.72,-0.05,0.15,0.30,-0.79,-0.80,-0.45,-0.23,-0.38,-0.91,-0.58
1,Eduardo Salvio,2018,Argentina,-0.42,-0.50,-0.51,0.64,0.02,-0.46,-0.05,0.48,0.61,0.67,0.32,-0.45,0.21,-1.11,0.68,-0.03
2,Enzo Pérez,2018,Argentina,-0.42,0.11,-0.51,-0.69,0.27,-0.65,-0.36,-0.34,0.35,-0.22,-0.80,-0.45,-0.89,0.72,0.04,-0.66
3,Federico Fazio,2018,Argentina,-0.42,-0.59,-0.51,-0.69,-0.97,-0.91,-0.67,-0.84,-0.82,-0.79,-0.64,-0.45,0.21,-1.11,-0.59,-0.77
4,Franco Armani,2018,Argentina,-0.42,-0.59,-0.51,-0.80,-0.67,0.64,-0.67,-0.67,-0.82,-0.79,-0.64,-0.45,-0.89,-1.11,-0.91,-0.77


In [19]:
df.sort_values(by = ['takeons_z'], ascending = False)

,player,season,team,goals_z,xg_z,crosses_z,boxtouches_z,passes_z,progpasses_z,takeons_z,progruns_z,tackles_z,interceptions_z,clearances_z,blocks_z,aerials_z,fouls_z,fouled_z,nsxg_z
4101,Diego Maradona,1986,Argentina,3.91,2.64,3.25,2.83,0.98,0.92,9.18,1.77,0.90,0.09,-0.71,-0.56,0.45,0.52,6.73,6.20
1142,Lionel Messi,2010,Argentina,-0.40,3.62,0.34,2.17,2.57,1.05,8.51,2.69,1.22,-0.37,-0.74,-0.48,-0.05,-0.37,2.72,4.14
5240,Alessandro Mazzola,1974,Italy,-0.41,2.37,2.97,1.61,1.85,1.85,7.27,6.14,-0.07,-0.72,-0.71,-0.54,-0.63,0.20,0.59,3.78
5400,Jairzinho,1970,Brazil,5.46,1.59,0.48,2.11,1.00,0.29,7.17,4.40,-0.18,-0.16,-0.52,-0.59,-0.20,1.06,3.83,2.64
8,Lionel Messi,2018,Argentina,1.30,3.08,0.66,1.30,1.17,0.32,6.76,2.12,2.15,-0.79,-0.80,-0.45,-0.89,0.35,4.49,4.55
588,Lionel Messi,2014,Argentina,2.85,1.34,2.01,1.05,0.68,0.24,6.75,3.23,0.72,-0.62,-0.72,-0.52,-0.81,-0.09,1.92,4.20
1205,Alexis Sánchez,2010,Chile,-0.40,1.69,0.55,4.44,0.01,-0.04,6.56,2.42,4.22,0.59,-0.50,-0.48,1.03,1.63,4.27,3.76
479,Isco,2018,Spain,1.17,1.24,5.41,1.55,4.36,0.98,6.47,7.34,0.47,1.19,-0.51,-0.45,-0.08,-0.77,4.67,4.44
38,Eden Hazard,2018,Belgium,2.53,3.05,-0.17,2.55,0.55,0.67,6.41,3.19,0.51,0.23,-0.71,-0.45,-0.14,0.14,3.99,3.31
2765,Ariel Ortega,1998,Argentina,1.90,1.07,0.78,1.56,1.10,0.44,6.01,2.67,0.13,-0.72,-0.66,-0.54,-0.75,0.35,6.05,4.32


In [20]:
df[df['player'] == 'Neymar']

,player,season,team,goals_z,xg_z,crosses_z,boxtouches_z,passes_z,progpasses_z,takeons_z,progruns_z,tackles_z,interceptions_z,clearances_z,blocks_z,aerials_z,fouls_z,fouled_z,nsxg_z
65,Neymar,2018,Brazil,2.33,5.02,4.15,5.82,1.30,0.91,4.78,4.28,0.39,-0.79,-0.80,0.17,-0.72,0.06,5.70,7.28
670,Neymar,2014,Brazil,3.15,1.65,2.55,1.44,-0.07,-0.24,2.32,1.29,-0.16,-0.56,-0.72,-0.03,-0.16,0.38,2.18,2.37


In [56]:
import requests
from bs4 import BeautifulSoup

In [63]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page1 = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1"
page2 = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/ajax/yw1/page/2"
page3 = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/ajax/yw1/page/3"
page4 = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/ajax/yw1/page/4"

In [66]:
pageTree = requests.get(page1, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
Players = pageSoup.find_all("a", {"class": "spielprofil_tooltip"})
Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})

In [67]:
PlayersList = []
ValuesList = []
for i in range(0,24):
    PlayersList.append(Players[i].text)
    ValuesList.append(Values[i].text)

df_prem_1 = pd.DataFrame({"Players":PlayersList, "Value":ValuesList})
df_prem_1.drop(df_prem_1.index[0], inplace = True)
df_prem_1

,Players,Value
1,Harry Kane,£135.00m
2,Mohamed Salah,£135.00m
3,Kevin De Bruyne,£108.00m
4,Eden Hazard,£90.00m
5,Romelu Lukaku,£90.00m
6,Dele Alli,£81.00m
7,Raheem Sterling,£81.00m
8,Leroy Sané,£81.00m
9,Paul Pogba,£72.00m
10,N'Golo Kanté,£72.00m


In [41]:
pageTree = requests.get(page2, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
Players = pageSoup.find_all("a", {"title": "spielprofil_tooltip"})
Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})

PlayersList = []
ValuesList = []
for i in range(0,26):
    PlayersList.append(Players[i].text)
    ValuesList.append(Values[i].text)

df_prem_2 = pd.DataFrame({"Players":PlayersList, "Value":ValuesList})
df_prem_2.drop(df_prem_2.index[0], inplace = True)
df_prem_2

,Players,Value
1,Virgil van Dijk,£45.00m
2,Ederson,£45.00m
3,Bernardo Silva,£45.00m
4,Riyad Mahrez,£45.00m
5,Aymeric Laporte,£45.00m
6,Jorginho,£45.00m
7,Heung-min Son,£45.00m
8,Kyle Walker,£40.50m
9,Mesut Özil,£40.50m
10,Aaron Ramsey,£40.50m


In [27]:
page = "https://www.transfermarkt.co.uk/fc-arsenal/startseite/verein/11/saison_id/2018"
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

Players2 = pageSoup.find_all("a", {"class": "spielprofil_tooltip"})
#Let's look at the first name in the Players list.
Players2[1].text

'B. Leno'

In [28]:
page2 = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1/ajax/yw1/page/2"
pageTree2 = requests.get(page2, headers=headers)
pageSoup2 = BeautifulSoup(pageTree2.content, 'html.parser')

Players3 = pageSoup2.find_all("a", {"class": "spielprofil_tooltip"})
#Let's look at the first name in the Players list.
Players3[0].text

'Mohamed Salah'

In [29]:
page3 = "https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985/saison_id/2018"
pageTree3 = requests.get(page3, headers=headers)
pageSoup3 = BeautifulSoup(pageTree3.content, 'html.parser')

Players3 = pageSoup3.find_all("a", {"class": "spielprofil_tooltip"})
#Let's look at the first name in the Players list.

In [30]:
# PlayersList = []
# ValuesList = []

# for i in range(1,50):
#     PlayersList.append(Players3[i].text)
#     ValuesList.append(Values[i].text)
    
# df = pd.DataFrame({"Players":PlayersList,"Values":ValuesList})
# df

In [31]:
page = 'https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1'
tree = requests.get(page, headers = headers)
soup = BeautifulSoup(tree.content, 'html.parser')

#Create an empty list to assign these values to
teamLinks = []

#Extract all links with the correct CSS selector
links = soup.select("a.vereinprofil_tooltip")

#We need the location that the link is pointing to, so for each link, take the link location. 
#Additionally, we only need the links in locations 1,3,5,etc. of our list, so loop through those only
for i in range(1,41,2):
    teamLinks.append(links[i].get("href"))
    
#For each location that we have taken, add the website before it - this allows us to call it later
for i in range(len(teamLinks)):
    teamLinks[i] = "https://www.transfermarkt.co.uk"+teamLinks[i]

In [32]:
teamLinks

['https://www.transfermarkt.co.uk/manchester-city/startseite/verein/281/saison_id/2018',
 'https://www.transfermarkt.co.uk/fc-chelsea/startseite/verein/631/saison_id/2018',
 'https://www.transfermarkt.co.uk/fc-chelsea/startseite/verein/631/saison_id/2018',
 'https://www.transfermarkt.co.uk/fc-liverpool/startseite/verein/31/saison_id/2018',
 'https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985/saison_id/2018',
 'https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985/saison_id/2018',
 'https://www.transfermarkt.co.uk/tottenham-hotspur/startseite/verein/148/saison_id/2018',
 'https://www.transfermarkt.co.uk/fc-arsenal/startseite/verein/11/saison_id/2018',
 'https://www.transfermarkt.co.uk/fc-arsenal/startseite/verein/11/saison_id/2018',
 'https://www.transfermarkt.co.uk/fc-everton/startseite/verein/29/saison_id/2018',
 'https://www.transfermarkt.co.uk/west-ham-united/startseite/verein/379/saison_id/2018',
 'https://www.transfermarkt.co.uk/west-ham-unit

In [33]:
#Create an empty list for our player links to go into
playerLinks = []

#Run the scraper through each of our 20 team links
for i in range(len(teamLinks)):

    #Download and process the team page
    page = teamLinks[i]
    tree = requests.get(page, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')

    #Extract all links
    links = soup.select("a.spielprofil_tooltip")
    
    #For each link, extract the location that it is pointing to
    for j in range(len(links)):
        playerLinks.append(links[j].get("href"))

    #Add the location to the end of the transfermarkt domain to make it ready to scrape
    for j in range(len(playerLinks)):
        playerLinks[j] = "https://www.transfermarkt.co.uk"+playerLinks[j]

    #The page list the players more than once - let's use list(set(XXX)) to remove the duplicates
    playerLinks = list(set(playerLinks))

In [34]:
playerLinks[1][558:]

''

In [35]:
playerLinks[657].rfind('https')

IndexError: list index out of range

In [36]:
len(playerLinks)

654

In [37]:
values = []
names = []
for i in range(600):

    #Take site and structure html
    idx = playerLinks[i].rfind('https')
    page = playerLinks[i][idx:]
    tree = requests.get(page, headers=headers)
    soup = BeautifulSoup(tree.content, 'html.parser')
    

    #Find image and save it with the player's name
    #Find the player's name
#     v = soup.find("div", attrs={"class":"right-td"})
#     value = v.text
#     values.append(value)
    name = pageSoup.find("h1", attrs = {"itemprop": "name"})
    names.append(name)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


ConnectionError: ('Connection aborted.', OSError("(54, 'ECONNRESET')",))

In [ ]:
PlayersList = []
ValuesList = []

for i in range(1,26):
#     PlayersList.append(Players[i].text)
    ValuesList.append(Values[i].text)
    
# df = pd.DataFrame({"Players":PlayersList, "Value":ValuesList})
df = pd.DataFrame({"Value": ValuesList})
df

In [79]:
prem_league_df = pd.read_csv('england-premier-league-players-2017-to-2018-stats.csv')

In [81]:
prem_league_df.columns

Index(['full_name', 'age', 'league', 'season', 'position',
       'minutes_played_overall', 'minutes_played_home', 'minutes_played_away',
       'nationality', 'appearances_overall', 'appearances_home',
       'appearances_away', 'goals_overall', 'goals_home', 'goals_away',
       'assists_overall', 'assists_home', 'assists_away', 'penalty_goals',
       'penalty_misses', 'clean_sheets_overall', 'clean_sheets_home',
       'clean_sheets_away', 'conceded_overall', 'conceded_home',
       'conceded_away', 'yellow_cards_overall', 'red_cards_overall',
       'goals_involved_per_90_overall', 'assists_per_90_overall',
       'goals_per_90_overall', 'goals_per_90_home', 'goals_per_90_away',
       'min_per_goal_overall', 'conceded_per_90_overall',
       'min_per_conceded_overall', 'min_per_match', 'min_per_card_overall',
       'min_per_assist_overall', 'cards_per_90_overall',
       'rank_in_league_top_attackers', 'rank_in_league_top_midfielders',
       'rank_in_league_top_defenders', 'ran

In [128]:
test3 = prem_league_df[['full_name','league']]
test3.columns = ['player_name','league']
test3

,player_name,league
0,Petr Čech,Premier League
1,Monreal,Premier League
2,Sead Kolašinac,Premier League
3,Hector Bellerin,Premier League
4,Rob Holding,Premier League
5,Mesut Özil,Premier League
6,Granit Xhaka,Premier League
7,Alex Oxlade-Chamberlain,Premier League
8,Mohamed Elneny,Premier League
9,Danny Welbeck,Premier League


In [87]:
start = time()
## Fetching data
#Connecting to database
path = "/Users/evanhosmer/galvanize/capstone-2/"  #Insert path here
database = path + 'database.sqlite'
conn = sqlite3.connect(database)

In [88]:
n_jobs = 1 #Insert number of parallel jobs here

#Fetching required data tables
player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match;", conn)

In [154]:
ligue = pd.read_csv('france-ligue-1-players-2017-to-2018-stats.csv')
newcol = ligue['full_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
ligue.drop(['full_name'], axis = 1, inplace = True)
t = pd.concat([ligue,newcol], axis = 1)
test4 = t[['league','full_name']]
test4.columns = ['league','player_name']

489

In [104]:
new = player_stats_data.merge(player_data, on = 'player_api_id')

In [147]:
test = new.groupby(['player_name']).mean()
test2 = test.reset_index()
test2[test2['player_name'] == 'Danijel Subasic']

,player_name,id_x,player_fifa_api_id_x,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,...,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,id_y,player_fifa_api_id_y,height,weight
2167,Danijel Subasic,36898.5,192593.0,115184.0,73.85,77.25,19.8,19.45,21.55,24.25,...,18.35,73.05,72.9,72.2,73.2,76.15,2206.0,192593.0,190.5,185.0


In [132]:
premier_league = test2.merge(test3, on = 'player_name')

In [155]:
ligue1 = test2.merge(test4, on = 'player_name')

,player_name,id_x,player_fifa_api_id_x,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,...,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,id_y,player_fifa_api_id_y,height,weight,league
